# Notebook walkthrough of using the Basic portion of Kiwiglider to process realtime Slocum glider data and create summary pages for delayed mode Slocum glider data

# Set up kiwiglider

You will need a C-compiler to use kiwiglider, since it relies on a package called dbdreader. See dbdreader installation suggestions for [Linux](https://github.com/smerckel/dbdreader?tab=readme-ov-file#installation-linux) and [Windows](https://github.com/smerckel/dbdreader?tab=readme-ov-file#installation-on-windows) for information about C-compilers (note that you do not need to install dbdreader itself at this stage)

We recommend creating a virtual environment.

1. `conda create -n kiwiglider`
1. `conda activate kiwiglider`

As of late May 2025, one dependency fails under pip but works with conda. As such, also do `conda install compliance-checker`

From your new virtual environment, you can now install kiwiglider. The easiest method as of late May 2025 is with `pip install -i https://test.pypi.org/simple/ kiwiglider`

Another method of installation is through git and pip. This will allow you to edit kiwiglider in a local repository. After you have [Git](https://git-scm.com/downloads) installed on your machine: 

1. Clone kiwiglider to your local machine: `git clone https://github.com/adeverne/kiwiglider`
1. Change to parent directory of kiwiglider
1. Install kiwiglider with `pip install -e ./kiwiglider`

# Load kiwiglider functions and classes

In [1]:
#general imports
from os.path import join as join_path
from os.path import exists
from os import sep,makedirs
from shutil import copy
import logging
#kiwiglider imports
from kiwiglider.basic import DeploymentYAML,DeploymentNetCDF

# Initialize

Start the logging package, which will keep us updated during processing

In [2]:
logging.basicConfig(level='INFO')

Specify the directory where all the organization's deployments and metadata live. In NIWA's case, and before running this notebook, the main directory is organized with


- deployments_NIWA.xlsx
- GLD0039
    - Raw
        - Cache
        - *.sbd
        - *.tbd
        - *.dbd
        - *.ebd
- GLD0040
    - Raw
        - Cache
        - *.sbd
        - *.tbd
        - *.dbd
        - *.ebd


and so on for all deployments

In [3]:
directory = join_path('C:',sep,'Users','elmerc','OneDrive - NIWA','Documents','SCRATCH')

Specify the deployment name and the summary to add in IOOS GliderDAC NetCDF files

In [4]:
deployment_ID = 40
deployment = 'GLD{:04d}'.format(deployment_ID)
summary = 'Slocum glider dataset gathered as part of the OCOF project at NIWA. This dataset contains physical oceanographic measurements of temperature, conductivity, salinity, density and estimates of depth-average currents.'

# Create YAML file for pyglider ingest

In [5]:
yml = DeploymentYAML(ID=deployment_ID)
yml.construct_yaml(excelsheet=join_path(directory,'deployments_NIWA.xlsx'),metadata={'summary':summary})
yml.write_yaml(outname=join_path(directory,deployment,'deployment_metadata.yml'))

INFO:kiwiglider.basic:Creating deployment YAML for 40
INFO:kiwiglider.basic:Getting metadata from C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\deployments_NIWA.xlsx
INFO:kiwiglider.basic:Adding global metadata to deployment YAML
INFO:kiwiglider.basic:Adding glider device metadata to deployment YAML
INFO:kiwiglider.basic:Adding variable metadata to deployment YAML
INFO:kiwiglider.basic:Adding profile variable metadata to deployment YAML
INFO:kiwiglider.basic:Adding QARTOD test metadata to deployment YAML
INFO:kiwiglider.basic:Wrting deployment YAML for 40 as C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\deployment_metadata.yml


We'll also copy this file into Realtime and Delayed directories. Kiwiglider is set up to ingest YAMLs from the same directory as the processed output files (allowing the user to specify different parameters for realtime and delayed modes), so if you wish to use the same YAML for realtime and delayed data it will need to be copied or you will need to specify it within the initialization of the DeploymentNetCDF class (below).

In [6]:
if not exists(join_path(directory,deployment,'Realtime')):
    makedirs(join_path(directory,deployment,'Realtime'))
copy(join_path(directory,deployment,'deployment_metadata.yml'),join_path(directory,deployment,'Realtime'))
if not exists(join_path(directory,deployment,'Delayed')):
    makedirs(join_path(directory,deployment,'Delayed'))
copy(join_path(directory,deployment,'deployment_metadata.yml'),join_path(directory,deployment,'Delayed'))

'C:\\Users\\elmerc\\OneDrive - NIWA\\Documents\\SCRATCH\\GLD0040\\Delayed\\deployment_metadata.yml'

# Create realtime files (eventually will send to IOOS GliderDAC)

This is the recommended use of kiwiglider basic, since kiwiglider steps contains a more thourough post-processing regiment for delayed mode data.

In this notebook, all realtime files are available. However, during normal use, this section will need to be run every time there is new data available.

## Initialize

In [7]:
realtime = DeploymentNetCDF(main_directory=join_path(directory,deployment))

## Create L0 (converted timeseries only)

While you can create individual profiles here (it is, in fact, the default) we will skip it for faster processing and smaller directory footprint.

In [8]:
realtime.make_L0('Realtime',l0profile_directory=None)

INFO:pyglider.slocum:C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Raw/*.[s|t]bd
INFO:pyglider.slocum:working on backscatter_470
INFO:pyglider.slocum:names: backscatter_470 sci_bb3slo_b470_scaled
INFO:pyglider.slocum:working on backscatter_532
INFO:pyglider.slocum:names: backscatter_532 sci_bb3slo_b532_scaled
INFO:pyglider.slocum:working on backscatter_660
INFO:pyglider.slocum:names: backscatter_660 sci_bb3slo_b660_scaled
INFO:pyglider.slocum:working on backscatter_700
INFO:pyglider.slocum:names: backscatter_700 sci_flbbcd_bb_units
INFO:pyglider.slocum:working on cdom
INFO:pyglider.slocum:names: cdom sci_flbbcd_cdom_units
INFO:pyglider.slocum:working on chlorophyll
INFO:pyglider.slocum:names: chlorophyll sci_flbbcd_chlor_units
INFO:pyglider.slocum:working on conductivity
INFO:pyglider.slocum:names: conductivity sci_water_cond
INFO:pyglider.slocum:working on heading
INFO:pyglider.slocum:names: heading m_heading
INFO:pyglider.slocum:working on latitude
INFO:pyglider.slocum:na

## Create L1 (with QARTOD tests)

In [9]:
realtime.make_L1()

INFO:kiwiglider.basic:Creating L1 single timeseries NetCDF C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Realtime\L1-timeseries\GLD0040.nc
INFO:kiwiglider.basic:Extracting QARTOD test parameters from C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\deployment_metadata.yml
INFO:kiwiglider.basic:Running QARTOD tests on C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Realtime\L0-timeseries/GLD0040.nc
INFO:ioos_qc.stores:Adding column time from stream backscatter_470
INFO:ioos_qc.stores:Adding column z from stream backscatter_470
INFO:ioos_qc.stores:Adding column lon from stream backscatter_470
INFO:ioos_qc.stores:Adding column lat from stream backscatter_470
INFO:kiwiglider.basic:Saving QARTOD test results as C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Realtime\L1-timeseries\GLD0040.nc
INFO:kiwiglider.basic:Creating L1 profile NetCDFs in C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Realtime\L1-profiles
INFO:pyglider.ncprocess:Extra

## Check IOOS GliderDAC compliance

In [10]:
realtime.check_compliance()

INFO:kiwiglider.basic:Checking profile NetCDFs in C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Realtime\L1-profiles
INFO:kiwiglider.basic:Checking C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Realtime\L1-profiles\betty591-20240306T2203_report.txt
INFO:kiwiglider.basic:C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Realtime\L1-profiles\betty591-20240306T2203.nc is IOOS GliderDAC compliant.See C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Realtime\L1-profiles\betty591-20240306T2203_report.txt for details.
INFO:kiwiglider.basic:Checking C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Realtime\L1-profiles\betty591-20240306T2256_report.txt
INFO:kiwiglider.basic:C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Realtime\L1-profiles\betty591-20240306T2256.nc is IOOS GliderDAC compliant.See C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Realtime\L1-profiles\betty591-20240306T2256_report.txt for details.
INFO:kiwiglid

# Create delayed mode summary

While not recommended for data disemination purposes, using kiwiglider basic represents a "quick and dirty" processing path. The main purpose for using kiwiglider basic is to produce a mission summary page, as follows.

## Initialize

In [11]:
delayed = DeploymentNetCDF(main_directory=join_path(directory,deployment))

## Create L0 (timeseries only)

While you can create individual profiles here (it is, in fact, the default) we will skip it for faster processing and smaller directory footprint.

In [12]:
delayed.make_L0('Delayed',l0profile_directory=None)

INFO:pyglider.slocum:C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Raw/*.[d|e]bd
INFO:pyglider.slocum:working on backscatter_470
INFO:pyglider.slocum:names: backscatter_470 sci_bb3slo_b470_scaled
INFO:pyglider.slocum:working on backscatter_532
INFO:pyglider.slocum:names: backscatter_532 sci_bb3slo_b532_scaled
INFO:pyglider.slocum:working on backscatter_660
INFO:pyglider.slocum:names: backscatter_660 sci_bb3slo_b660_scaled
INFO:pyglider.slocum:working on backscatter_700
INFO:pyglider.slocum:names: backscatter_700 sci_flbbcd_bb_units
INFO:pyglider.slocum:working on cdom
INFO:pyglider.slocum:names: cdom sci_flbbcd_cdom_units
INFO:pyglider.slocum:working on chlorophyll
INFO:pyglider.slocum:names: chlorophyll sci_flbbcd_chlor_units
INFO:pyglider.slocum:working on conductivity
INFO:pyglider.slocum:names: conductivity sci_water_cond
INFO:pyglider.slocum:working on heading
INFO:pyglider.slocum:names: heading m_heading
INFO:pyglider.slocum:working on latitude
INFO:pyglider.slocum:na

## Create summary page

In [14]:
delayed.create_summary(
    timeseries_file=join_path('Delayed','L0-timeseries',deployment+'.nc'),
    output_file=deployment+'.png',
    author='Cassandra Elmer'
)

INFO:kiwiglider.basic:Creating C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Delayed\GLD0040.png
INFO:kiwiglider.basic:Reading C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Delayed\L0-timeseries\GLD0040.nc into memory
INFO:kiwiglider.basic:Adding Density (kg m-3) plot to summary page
INFO:kiwiglider.basic:Adding water salinity (1e-3) plot to summary page
INFO:kiwiglider.basic:Adding Temperature (Celsius) plot to summary page
INFO:kiwiglider.basic:Adding map to summary page
grdimage [WARNING]: Longitude range too small; geographic boundary condition changed to natural.
INFO:kiwiglider.basic:Adding metadata snapshot to summary page
INFO:kiwiglider.basic:Adding header information to summary page
INFO:kiwiglider.basic:Saving summary page as C:\Users\elmerc\OneDrive - NIWA\Documents\SCRATCH\GLD0040\Delayed\GLD0040.png
